# Project dependencies

In [1]:
# Load .env variables
from dotenv import load_dotenv
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
# Cassandra), and OpenAI (to generate embeddings)
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# These are used to authenticate with Astra DB
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

import json
import os 

In [4]:

load_dotenv()

# get Astra connection information from environment variables
ASTRA_DB_ID = os.environ.get('ASTRA_DB_ID')
print('ASTRA_DB_ID: ', ASTRA_DB_ID)
ASTRA_DB_REGION = os.environ.get('ASTRA_DB_REGION')
ASTRA_DB_APPLICATION_TOKEN = os.environ.get('ASTRA_DB_APPLICATION_TOKEN')
ASTRA_DB_KEYSPACE = os.environ.get('ASTRA_DB_KEYSPACE')
ASTRA_DB_NAME = os.environ.get('ASTRA_DB_NAME')
print('ASTRA_DB_KEYSPACE: ', ASTRA_DB_KEYSPACE)
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# get cwd
path_to_root = os.getcwd()
ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(path_to_root, 'secure-connect-'+ASTRA_DB_NAME+'-db'+'.zip')
print('ASTRA_DB_SECURE_BUNDLE_PATH: ', ASTRA_DB_SECURE_BUNDLE_PATH)

# Set up the OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

ASTRA_DB_ID:  ce7900d0-52c3-4f86-97fb-ecc13ab5c952
ASTRA_DB_KEYSPACE:  mindful
ASTRA_DB_SECURE_BUNDLE_PATH:  c:\Users\stu-kalebnim\Documents\GitHub\MindfulHacks_23\secure-connect-mindfullhacks-db.zip


## Test Connection + Add document

In [3]:
from astrapy.rest import create_client, http_methods
import uuid
import os

# get Astra connection information from environment variables
ASTRA_DB_ID = os.environ.get('ASTRA_DB_ID')
ASTRA_DB_REGION = os.environ.get('ASTRA_DB_REGION')
ASTRA_DB_APPLICATION_TOKEN = os.environ.get('ASTRA_DB_APPLICATION_TOKEN')
ASTRA_DB_KEYSPACE = os.environ.get('ASTRA_DB_KEYSPACE')

COLLECTION_NAME = "test"

# setup an Astra Client
astra_http_client = create_client(astra_database_id=ASTRA_DB_ID,
  astra_database_region=ASTRA_DB_REGION,
  astra_application_token=ASTRA_DB_APPLICATION_TOKEN)

# create a document on Astra using the Document API
doc_uuid = uuid.uuid4()
print(f'Sending PUT: /api/rest/v2/namespaces/{ASTRA_DB_KEYSPACE}/collections/{COLLECTION_NAME}/{doc_uuid}')

response = astra_http_client.request(
  method=http_methods.PUT,
  path=f"/api/rest/v2/namespaces/{ASTRA_DB_KEYSPACE}/collections/{COLLECTION_NAME}/{doc_uuid}",
  json_data={
    "first_name": "Cliff",
    "last_name": "Wicklow",
    "emails": ["cliff.wicklow@example.com"],
})

print (response)

Sending PUT: /api/rest/v2/namespaces/mindful/collections/test/1c1a338d-7b62-44fe-bd9f-2f78af64c173
{'documentId': '1c1a338d-7b62-44fe-bd9f-2f78af64c173'}


## Connecting to cluster

In [5]:
cloud_config= {
  "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH
}
auth_provider=PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
astraSession = cluster.connect()

DriverException: Unable to connect to the metadata service at https://ce7900d0-52c3-4f86-97fb-ecc13ab5c952-us-east1.db.astra.datastax.com:29080/metadata. Check the cluster status in the cloud console. 

# Test 

In [3]:
from llm.chains import eventDetails
output = eventDetails.run(transcript = "Im here to help. Whats going on?I e upwith my girlfriend recentlyOh, I'm so sorry to hear that, that can be really tough. Want to talk about it? Im happy to lend an ear. Was it a mutual decision or a difficult break up?It was quite difficult since I really did not want to let her goBreakups are always hard, but especially when it's not your choice. Can I ask what happened? You dont have to share if its too personal, I just want to be there for you if you need to talk.")

ValidationError: 1 validation error for Generation
text
  none is not an allowed value (type=type_error.none.not_allowed)

In [2]:
from langchain.llms import OpenAI

eventDetails_llm = OpenAI(model_name="gpt-4-0613", temperature=1)
eventDetails_llm("Tell me a joke")

"Why don't scientists ever trust atoms?\n\nBecause they make up everything!"